In [86]:
import numpy as np
import pandas as pd
import os
import sys

from subprocess import call, check_output, STDOUT, check_call

# Using `partis` to generate synthetic datasets

We can use the software `partis` to generate synthetic datasets. (**Add link to partis**)

In [87]:
# configuration
partis_path = '/home/fede/src/partis_old'
output_path = '/home/fede/projects_local/davide/data/partis_RAW_v6'
list_number_clones = [100, 500, 1500, 3000, 5000]
n_iter = 3

In [ ]:
# need to install geiger, ape, TreeSim in R for this to work
for i in list_number_clones:
    for j in range(0, 2):
        call('{1}/bin/partis simulate '
             #'--parameter-dir {1}/test/reference-results/test/parameters/data/ '
             '--simulate-partially-from-scratch '
             '--outfname {3}/clones_{0}.{2}.csv --n-sim-events {0} --n-leaves 200 '
             '--indel-frequency 0.05 --indel-location cdr3 --mean-indel-length 6 '
             '--n-procs 16'.format(i, partis_path, j, output_path).split())

This produces into `output_path` folder a list of RAW sequences.

To simplify the processing of IMGT/HighV-Quest, let's a unique `fasta` file where, in the `ID` string, there is also the identity of the original database name. This will allow us to recover our original databases splitted.

In [16]:
path = output_path + '/'
files = [path + x for x in os.listdir(path) if x.endswith('.csv')]

# 1. create a single pandas dataframe
db_s = []
for x in files:
    df = pd.read_csv(x, index_col=None)
    df['db'] = x.split('/')[-1]
    db_s.append(df)

df = pd.concat(db_s)

In [17]:
# 2. create fasta file up to 500k sequences
for i in range(df.shape[0] / 500000 + 1):
    with open(os.path.join(path, "all_{}.fasta".format(i)), 'w') as f:
        for index, row in (df.iloc[i * 500000:(i+1)*500000].iterrows()):
            f.write(">" + "_".join([row['db']] + [str(a) for a in row.values[:-8]]))
            f.write("\n")
            f.write(row['seq'])
            f.write("\n")

Ok! Now we can use IMGT to convert our `fasta` file(s) into databases which we can use as input to ICING.
To do so, connect to IMGT HighV-Quest software and upload the data.

When finished, an email will notify that results are ready. Now, download them and extract the "txz" files as folders to use them with Change-O `MakeDb` script.

In [8]:
%%bash
# run Changeo to convert IMGT into fasta file
# python MakeDb.py imgt -i <imgt output, zip or folder> -s <original fasta file> --scores
for i in {1..12}
  do 
     python /home/fede/Dropbox/projects/davide/changeo/MakeDb.py imgt -i /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_$i -s /home/fede/projects_local/davide/data/partis_RAW_v5/fasta/all_$i.fasta --scores
done

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_1
     SEQ_FILE> all_1.fasta
     NO_PARSE> False
 SCORE_FIELDS> True

PROGRESS> 15:54:57 [####################] 100% (500,000) 2.4 min

OUTPUT> /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_1_db-pass.tab
  PASS> 481377
  FAIL> 18623
   END> MakeDb

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_2
     SEQ_FILE> all_2.fasta
     NO_PARSE> False
 SCORE_FIELDS> True

PROGRESS> 15:57:33 [####################] 100% (500,000) 2.4 min

OUTPUT> /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_2_db-pass.tab
  PASS> 483892
  FAIL> 16108
   END> MakeDb

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v5/imgt-processed/partis_indel_3
     SEQ_FILE> all_

Divide now the IMGT-ChangeO processed files into a final list of databases which are usable from our method.

In [11]:
path = '/home/fede/projects_local/davide/data/partis_RAW_v5/makedb-pass/'
db_s = []
for f in [os.path.join(path, x) for x in os.listdir(path) if x.endswith('db-pass.tab')]:
    db_s.append(pd.read_csv(f, dialect='excel-tab'))

df = pd.concat(db_s)
    
# add the mut column
df['MUT'] = (1 - df['V_IDENTITY']) * 100.

df['DB'] = df['SEQUENCE_ID'].str.split('.csv').apply(lambda x: min(x, key=len))
for i in df.DB.unique():
    df[df.DB == i].to_csv(os.path.join(path, str(i) + '.tab'), index=False, sep='\t')

Let's produce an overview of the datasets.

In [60]:
from icing.utils import io
df_all = pd.DataFrame()

for f in [os.path.join(path, x) for x in os.listdir(path) if x.startswith('clones_')]:
    df = io.load_dataframe(f)
    df['true_clone'] = [x[3] for x in df.sequence_id.str.split('_')] 
    row = {}
    row['database'] = f.split('/')[-1]
    row['n_seqs'] = int(df.shape[0])
    row['clonotypes'] = int(df.true_clone.unique().size)
    row['avg seqs/clone'] = np.mean([len(x) for x in df.groupby('true_clone').groups.values()])
    
    df['true_v'] = [parseAllele(x[4], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
    row['unique V genes'] = int(df.true_v.unique().size)
    df['true_d'] = [parseAllele(x[5], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
    row['unique D genes'] = int(df.true_d.unique().size)
    df['true_j'] = [parseAllele(x[6], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
    row['unique J genes'] = int(df.true_j.unique().size)
    
    row['mean (std) of V gene mutation'] = "%.2f (%.2f)" % (df.mut.mean(), df.mut.std())
    df_all = df_all.append(row, ignore_index=True)

In [16]:
from icing.utils import io
df = io.load_dataframe('/home/fede/projects_local/davide/data/partis_RAW_v5/makedb-pass/clones_100.0.tab')
df['true_clone'] = [x[3] for x in df.sequence_id.str.split('_')] 

/home/fede/miniconda2/lib/python2.7/site-packages/Bio/Seq.py:2071: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [65]:
from icing.externals.DbCore import parseAllele, gene_regex, junction_re
# df['true_v'] = [parseAllele(x[4], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
# print df.true_v.unique().size
# df['true_d'] = [parseAllele(x[5], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
# print df.true_d.unique().size
df['true_j'] = [parseAllele(x[5], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
print df.true_j.unique()

"%.2f (%.2f)" % (df.mut.mean(), df.mut.std())

['IGHD3-16' 'IGHD6-19' 'IGHD3-10' 'IGHD6-6' 'IGHD3-22' 'IGHD2-2' 'IGHD2-15'
 'IGHD2-8' 'IGHD2-21']


'12.06 (10.00)'

In [84]:
df_all['indexNumber'] = [int(i.split('_')[-1].split('.')[0]) + int(
    i.split('_')[-1].split('.')[1]) for i in df_all.database]
# Perform sort of the rows
df_all.sort_values(['indexNumber'], ascending = [True], inplace = True)
# Deletion of the added column
df_all.drop('indexNumber', 1, inplace = True)

df_all['avg seqs/clone'] = df_all['avg seqs/clone'].map('{:.2f}'.format)

df_all[['n_seqs', 'clonotypes', 'unique V genes', 'unique D genes', 'unique J genes']] = df_all[
    ['n_seqs', 'clonotypes', 'unique V genes', 'unique D genes', 'unique J genes']].astype(int)

sorted_df = df_all.loc[:, ['database', 'n_seqs', 'clonotypes', 'avg seqs/clone', 'unique V genes',
               'unique D genes', 'unique J genes', 'mean (std) of V gene mutation']]

In [85]:
sorted_df.to_latex("/home/fede/Dropbox/projects/icing/cibb17/dataset_table.tex", index=False)